In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
os.getcwd()

'H:\\pyspark_advanced-coding_interview'

# Incremental data loading

In [4]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("IncrementalDataLoad").getOrCreate()

# Sample Data for Existing (Target) Table
target_data = [
    (1, "Alice", 5000, "Active"),
    (2, "Bob", 6000, "Active"),
    (3, "Charlie", 7000, "Inactive"),
    (4, "David", 5500, "Active"),
    (5, "Eva", 6500, "Inactive")
]

# Sample Data for New (Source) Table
source_data = [
    (3, "Charlie", 7500, "Active"),  # Update existing record
    (5, "Eva", 7000, "Active"),      # Update existing record
    (6, "Frank", 8000, "Active"),    # Insert new record
    (7, "Grace", 7200, "Active")     # Insert new record
]

# Create DataFrames
target_df = spark.createDataFrame(target_data, ["id", "name", "salary", "status"])
source_df = spark.createDataFrame(source_data, ["id", "name", "salary", "status"])

# Create Temporary Views for Spark SQL
target_df.createOrReplaceTempView("target_table")
source_df.createOrReplaceTempView("source_table")

# Show the Original DataFrames
print("Target Table:")
target_df.show()

print("Source Table:")
source_df.show()




Target Table:
+---+-------+------+--------+
| id|   name|salary|  status|
+---+-------+------+--------+
|  1|  Alice|  5000|  Active|
|  2|    Bob|  6000|  Active|
|  3|Charlie|  7000|Inactive|
|  4|  David|  5500|  Active|
|  5|    Eva|  6500|Inactive|
+---+-------+------+--------+

Source Table:
+---+-------+------+------+
| id|   name|salary|status|
+---+-------+------+------+
|  3|Charlie|  7500|Active|
|  5|    Eva|  7000|Active|
|  6|  Frank|  8000|Active|
|  7|  Grace|  7200|Active|
+---+-------+------+------+



In [5]:
target_df.cache()
source_df.cache()

res = spark.sql("""
                
MERGE INTO target_table AS t
USING source_table AS s
ON t.id = s.id
WHEN MATCHED THEN
  UPDATE SET t.name = s.name, t.salary = s.salary, t.status = s.status
WHEN NOT MATCHED THEN
  INSERT (id, name, salary, status) VALUES (s.id, s.name, s.salary, s.status);
             
                
                """)

res.show()

Py4JJavaError: An error occurred while calling o24.sql.
: org.apache.spark.SparkUnsupportedOperationException: MERGE INTO TABLE is not supported temporarily.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.ddlUnsupportedTemporarilyError(QueryExecutionErrors.scala:1109)
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$.apply(SparkStrategies.scala:898)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:70)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(Trave

In [7]:
#--Step 1: Update existing records
#-- Step 2: Insert new records


res1 = spark.sql(""" 
  

UPDATE target_table
SET name = s.name, salary = s.salary, status = s.status
FROM source_table AS s
WHERE target_table.id = s.id;


INSERT INTO target_table
SELECT s.*
FROM source_table s
WHERE NOT EXISTS (SELECT 1 FROM target_table t WHERE t.id = s.id);
               
                 
                 """)
res1.show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'FROM'.(line 6, pos 0)

== SQL ==
 
  

UPDATE target_table
SET name = s.name, salary = s.salary, status = s.status
FROM source_table AS s
^^^
WHERE target_table.id = s.id;


INSERT INTO target_table
SELECT s.*
FROM source_table s
WHERE NOT EXISTS (SELECT 1 FROM target_table t WHERE t.id = s.id);
               
                 
                 


In [8]:
from pyspark.sql.functions import col

# Step 1: Identify records to update and delete them
updates_df = source_df.join(target_df, "id", "inner").select(source_df["*"])

# Step 2: Identify records to insert
inserts_df = source_df.join(target_df, "id", "leftanti")

# Step 3: Create the updated target DataFrame
target_updated_df = target_df.alias("t").join(updates_df.alias("u"), "id", "leftanti").union(inserts_df).union(updates_df)

# Show the final DataFrame after upsert
print("Final Target Table After Upsert:")
target_updated_df.show()


Final Target Table After Upsert:
+---+-------+------+------+
| id|   name|salary|status|
+---+-------+------+------+
|  1|  Alice|  5000|Active|
|  2|    Bob|  6000|Active|
|  4|  David|  5500|Active|
|  6|  Frank|  8000|Active|
|  7|  Grace|  7200|Active|
|  3|Charlie|  7500|Active|
|  5|    Eva|  7000|Active|
+---+-------+------+------+



In [9]:
from delta.tables import DeltaTable

# Convert target DataFrame to Delta Table (if not already)
target_df.write.format("delta").save("/tmp/delta/target_table")
source_df.write.format("delta").save("/tmp/delta/source_table")

# Create DeltaTable object
delta_target = DeltaTable.forPath(spark, "/tmp/delta/target_table")

# Use Delta Lake merge to perform the upsert
delta_target.alias("t").merge(
    source=source_df.alias("s"),
    condition="t.id = s.id"
).whenMatchedUpdate(set={
    "name": "s.name",
    "salary": "s.salary",
    "status": "s.status"
}).whenNotMatchedInsert(values={
    "id": "s.id",
    "name": "s.name",
    "salary": "s.salary",
    "status": "s.status"
}).execute()


ModuleNotFoundError: No module named 'delta'